# 過去5日間以内の値動きとその後の値動きの間の相関を探す
- ラリー・ウィリアムズの短期売買法には、**「トレーディングやシステム開発で最高の結果が得られたのは、ほとんどの場合、過去1日から過去4日間の平均を使った時だったのである。」**と書かれていたから、過去4日ぐらいまで見てみるのが良いかもしれない。
- 大目に見て過去5日までの値動きを考慮可能にしておく。

### 仮説
- 過去5日以内の値動き・指標をもとに、その後の値動き（上昇・下降）をランダムより十分高い確率で予測できる。

### 検証すること
- 過去N日間の安値を下回ってから上昇に転じ、前日の高値を上回ったときにその後の値動きがどうなるか。（ラリー本によれば上昇する）

### 結果概要
- セットアップとして使えるほどの相関を見出すことはできなかった。詳細は下のほうに。

In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import locale

# 月や曜日を英語で取得するためこの設定をしておく
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

'en_US.UTF-8'

In [2]:
df_raw = pd.read_csv('./data/N225minif_2020_exchange_daily.csv')
df_raw.info()
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3377 entries, 0 to 3376
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   日付      3377 non-null   object
 1   始値      3377 non-null   int64 
 2   高値      3377 non-null   int64 
 3   安値      3377 non-null   int64 
 4   終値      3377 non-null   int64 
 5   出来高     3377 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 158.4+ KB


,日付,始値,高値,安値,終値,出来高
0,2006/7/18,14680,14715,14400,14400,22345
1,2006/7/19,14500,14650,14465,14560,20646
2,2006/7/20,14860,14975,14780,14975,17398
3,2006/7/21,14840,14875,14790,14820,14689
4,2006/7/24,14685,14870,14560,14775,22641


# 分析するためのデータを作成
- 過去N日間の価格データを使えるようにする

In [3]:
# 過去N日間のデータを扱えるようにする
lastN = 5

# 元のデータを最も古い日（＝N日前）として扱う
df = df_raw.rename(
    columns={'日付': 'date', '始値': f'o{lastN}', '高値': f'h{lastN}', '安値': f'l{lastN}', '終値': f'c{lastN}', '出来高': f'v{lastN}'}
)

for i in reversed(range(lastN)):
    inc = lastN - i
    df[f'o{i}'] = df[f'o{lastN}'][inc:].append(pd.Series([np.nan]*inc)).reset_index(drop=True)
    df[f'h{i}'] = df[f'h{lastN}'][inc:].append(pd.Series([np.nan]*inc)).reset_index(drop=True)
    df[f'l{i}'] = df[f'l{lastN}'][inc:].append(pd.Series([np.nan]*inc)).reset_index(drop=True)
    df[f'c{i}'] = df[f'c{lastN}'][inc:].append(pd.Series([np.nan]*inc)).reset_index(drop=True)
    df[f'v{i}'] = df[f'v{lastN}'][inc:].append(pd.Series([np.nan]*inc)).reset_index(drop=True)

# 日付がN日分ずれているので補正
df['date'] = df['date'][lastN:].append(pd.Series(['-']*lastN)).reset_index(drop=True)

# 過去N日分のデータを使うため、最後のN日間のデータは欠損値が発生する
#（上記の処理でとりあえずnp.nanと'-'で埋めている）
# したがって最後のN日間のデータを削除
df = df[:-lastN]

# 当日と1日前が上昇だったか下降だったかを見るので（終値 - 始値）を計算しておく
df['CODiff0'] = df['c0'] - df['o0']
df['CODiff1'] = df['c1'] - df['o1']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3372 entries, 0 to 3371
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     3372 non-null   object 
 1   o5       3372 non-null   int64  
 2   h5       3372 non-null   int64  
 3   l5       3372 non-null   int64  
 4   c5       3372 non-null   int64  
 5   v5       3372 non-null   int64  
 6   o4       3372 non-null   float64
 7   h4       3372 non-null   float64
 8   l4       3372 non-null   float64
 9   c4       3372 non-null   float64
 10  v4       3372 non-null   float64
 11  o3       3372 non-null   float64
 12  h3       3372 non-null   float64
 13  l3       3372 non-null   float64
 14  c3       3372 non-null   float64
 15  v3       3372 non-null   float64
 16  o2       3372 non-null   float64
 17  h2       3372 non-null   float64
 18  l2       3372 non-null   float64
 19  c2       3372 non-null   float64
 20  v2       3372 non-null   float64
 21  o1       3372 

# 過去N日間の安値を下回ってから上げて引けたとき、その次の日に上昇するか
- この分析は、過去N－１日間までしかできない。なぜなら最後に**翌日**の値動きを観察するから

### 結果

#### 過去3日間
- 過去3日間の安値より安い安値の日の発生確率は28.3%
- 過去3日間の安値より安い安値の日に上げて引けた日の発生確率は7.7%
- 過去3日間の安値より安い安値の日に上げて引けた時、その翌日に上昇する確率は47.9%
- 過去3日間の安値より安い安値の日に上げて引けてかつその前日の高値より高い高値だった（アウトサイドデイ）時、その翌日に上昇する確率は44.6%

#### 過去4日間
- 過去4日間の安値より安い安値の日の発生確率は24.6%（データ期間中222日）
- 過去4日間の安値より安い安値の日に上げて引けた日の発生確率は6.6%
- 過去4日間の安値より安い安値の日に上げて引けた時、その翌日に上昇する確率は47.3%
- 過去4日間の安値より安い安値の日に上げて引けてかつその前日の高値より高い高値だった（アウトサイドデイ）時、その翌日に上昇する確率は43.2%

#### まとめ
- さすがに最後の条件などは上昇確率がある程度高いかと思ったけどむしろ低かった。計算に誤りがあることも疑ってしまうが、いずれにしてもこのように闇雲に上昇セットアップを探すやり方はどう考えても非効率だ。
- そろそろ機械学習に手を出すか。というか、まずはいろんな指標を片っ端からDFに突っ込んでseaborn.pairplot()をするなどして相関を見てみればよいと思う。それで強い相関が見いだせればよいけど、あまりで無さそうだったら、機械学習に進んだほうが良いかもしれない。ランダムフォレストを使えば各特徴量の寄与度が分かりやすいから、単に強い寄与をする特徴量を探すためだけにやるというのもありだろう。

In [13]:
# 過去N日間の安値
nDays = 3 # 指定できるのは (lastN - 1) まで

df[f'low{nDays}'] = df[[f'l{i}' for i in range(2, nDays+2)]].min(axis=1)

In [14]:
# 過去N日間の安値より低い安値の日
df_low = df[df['l1'] < df[f'low{nDays}']]
df_low.info()
print(f'全日数：{df.shape[0]}、過去{nDays}日間の安値より低い安値の日：{df_low.shape[0]}')
print(f'p(過去{nDays}日間の安値より低い安値の日) = {100 * df_low.shape[0] / df.shape[0]:.1f}%')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 953 entries, 10 to 3366
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     953 non-null    object 
 1   o5       953 non-null    int64  
 2   h5       953 non-null    int64  
 3   l5       953 non-null    int64  
 4   c5       953 non-null    int64  
 5   v5       953 non-null    int64  
 6   o4       953 non-null    float64
 7   h4       953 non-null    float64
 8   l4       953 non-null    float64
 9   c4       953 non-null    float64
 10  v4       953 non-null    float64
 11  o3       953 non-null    float64
 12  h3       953 non-null    float64
 13  l3       953 non-null    float64
 14  c3       953 non-null    float64
 15  v3       953 non-null    float64
 16  o2       953 non-null    float64
 17  h2       953 non-null    float64
 18  l2       953 non-null    float64
 19  c2       953 non-null    float64
 20  v2       953 non-null    float64
 21  o1       953 n

In [15]:
# 過去N日間の安値より安い安値の日に、上げて引けた日
df_low_up = df_low[df_low['CODiff1'] > 0]
print(f'過去{nDays}日間の安値より安い安値の日に上げて引けた日数：{df_low_up.shape[0]}')
print(f'p(過去{nDays}日間の安値より安い安値の日に上げて引けた日) = {100*df_low_up.shape[0]/df.shape[0]:.1f}%')

過去3日間の安値より安い安値の日に上げて引けた日数：259
p(過去3日間の安値より安い安値の日に上げて引けた日) = 7.7%


In [16]:
# 過去N日間の安値より安い安値の日に上げて引けた場合、その翌日は上昇するか
df_low_upx2 = df_low_up[df_low_up['CODiff0'] > 0]
print(f'過去{nDays}日間の安値より安い安値の日に上げて引けた日数：{df_low_up.shape[0]}')
print(f'その翌日に上げて引けた日数：{df_low_upx2.shape[0]}')
print(f'p(翌日上昇|前日上昇) = {100 * df_low_upx2.shape[0] / df_low_up.shape[0]:.1f}%')

過去3日間の安値より安い安値の日に上げて引けた日数：259
その翌日に上げて引けた日数：124
p(翌日上昇|前日上昇) = 47.9%


In [17]:
# 過去N日間の安値より安い安値の日に上げて引けてかつ前日の高値より高い高値となった場合、その翌日は上昇するか
df_low_up_h = df_low_up[df_low_up['h1'] > df_low_up['h2']]
df_low_up_h_up = df_low_up_h_up = df_low_up_h[df_low_up_h['CODiff0'] > 0]
print(f'過去{nDays}日間の安値より安い安値の日に上げて引けてかつ前日より高い高値だった日数：{df_low_up_h.shape[0]}')
print(f'その翌日に上げて引けた日数：{df_low_up_h_up.shape[0]}')
print(f'p(翌日上昇|前日上昇) = {100 * df_low_up_h_up.shape[0] / df_low_up_h.shape[0]:.1f}%')

過去3日間の安値より安い安値の日に上げて引けてかつ前日より高い高値だった日数：56
その翌日に上げて引けた日数：25
p(翌日上昇|前日上昇) = 44.6%
